**Jose Miguel Acuña Hernandez** 

**jacunah@unal.edu.co**

En este documento constato como se realizo la limpieza de los datos entregados en el documento *8. Respuestas Acoso Escoral.xlsx - Respuestas de formulario.csv*

# Llamado delibrerias y base de datos 

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data_prepros = pd.read_csv('/root/Proyectos/DataAnalysisJuanDeCastellanos/data/8. Respuestas Acoso Escolar.xlsx - Respuestas de formulario 1.csv')

# Cambiando nombres de las columnas a unos mas razonables

In [3]:
data_prepros = data_prepros.rename(columns = {'Marca temporal' : 'Marca temporal', 'Ciudad ' : 'Ciudad', ' Colegio' : 'Colegio', 'Grado' : 'Grado', 'Edad' : 'Edad', 'Género ' : 'Género',
       'Grupo_familiar' : 'Grupo familiar', 'Zona_Vivienda' : 'Zona de vivienda', 'Grupo_Social' : 'Grupo social',
       '8  Empujando_intencional' : 'Empuje intencional', ' 9. Halado_cabello_agredido' : 'Halon de cabello',
       '10 Agredido_con_golpes ' : 'Golpes', '11 Dañado_objetos ' : 'Dañar objetos',
       '12. ¿Lanzan_objetos? ' : 'Lanzar objetos', '13. ¿Insultos_ofensas?' : 'Insultos',
       '14 ¿Víctima_chantaje?' : 'Chismes', '15 ¿Avergonzado_público?' : 'Avergonzado publicamente', '16 ¿Amenazas? ' : 'Amenazas',
       '17 ¿Víctima_bromas_pesadas?' : 'Bromas pesadas', '18 ¿Frases excluyentes?' : 'Frases excluyentes',
       '19 ¿Burlas_por_aspecto_físico_o_ condiciones_ particulares? ' : 'Burlas por aspecto o condición',
       '20 ¿Mensajes_contenido_ofensivo_desagradable?' : 'Mensajes ofensivos o desagradables',
       '21. ¿Insultos, groserías y palabras desagradables a través de redes sociales? ' : 'Insultos por redes sociales',
       '22. ¿Ha recibido fotos y/o videos desagradables o incómodos sobre usted u otras personas a través de las redes sociales?' : 'Fotos o videos incomodos en redes sociales',
       '23. ¿Ha recibido chantaje o amenazas por fotos o videos incómodos donde usted aparezca?' : 'Chantaje por fotos o videos suyos en redes sociales'})

# Eliminando datos innecesarios y valores nulos

In [4]:
data_prepros = data_prepros.drop(columns=['Marca temporal', 'Edad', 'Zona de vivienda', 'Grupo social', 'Grado'])


In [5]:
data_prepros = data_prepros[data_prepros['Ciudad'] == 'Tunja']
data_prepros = data_prepros.drop(columns=['Ciudad'])

In [6]:
data_prepros = data_prepros.dropna()

# Conversión de valores cualitativos a cuantitativos

In [7]:
valores_a_reemplazar = {
    'Nunca': 0,
    'Una vez al mes': 1,
    'Entre 2 y 4 veces al mes': 2,
    '5 o más veces al mes': 5,
}

columnas = ['Empuje intencional', 'Halon de cabello', 'Golpes', 'Dañar objetos', 'Lanzar objetos', 'Insultos', 'Chismes', 'Avergonzado publicamente', 'Amenazas', 'Bromas pesadas', 'Frases excluyentes', 'Burlas por aspecto o condición', 'Mensajes ofensivos o desagradables', 'Insultos por redes sociales', 'Fotos o videos incomodos en redes sociales', 'Chantaje por fotos o videos suyos en redes sociales']

for columna in columnas:
    data_prepros[columna] = data_prepros[columna].replace(valores_a_reemplazar)

Tome Nunca y lo remplace por 0, Una vez al mes por 1, Entre 2 y 4 veces al mes por 2 y 5 o más veces al mes por 5. Si me lo piden, puedo hacer una asignación distinta, sin embargo considero que esta es la mas lógica.

# Conversión de strings a floats

In [8]:
columnas = data_prepros.columns[3:]

# Convierte esas columnas a int
for columna in columnas:
    data_prepros[columna] = data_prepros[columna].apply(pd.to_numeric, errors='coerce')

In [9]:
data_prepros['Reportes de abuso físico'] = data_prepros[['Empuje intencional', 'Halon de cabello', 'Golpes', 'Dañar objetos', 'Lanzar objetos']].sum(axis=1)
data_prepros['Reportes de abuso psicológico'] = data_prepros[['Insultos', 'Chismes', 'Avergonzado publicamente', 'Amenazas', 'Bromas pesadas', 'Frases excluyentes', 'Burlas por aspecto o condición']].sum(axis=1)
data_prepros['Reportes de ciberbullying'] = data_prepros[['Mensajes ofensivos o desagradables', 'Insultos por redes sociales', 'Fotos o videos incomodos en redes sociales', 'Chantaje por fotos o videos suyos en redes sociales']].sum(axis=1)

In [10]:
Datos = data_prepros.drop(columns=['Empuje intencional', 'Halon de cabello', 'Golpes', 'Dañar objetos', 'Lanzar objetos', 'Insultos', 'Chismes', 'Avergonzado publicamente', 'Amenazas', 'Bromas pesadas', 'Frases excluyentes', 'Burlas por aspecto o condición', 'Mensajes ofensivos o desagradables', 'Insultos por redes sociales', 'Fotos o videos incomodos en redes sociales', 'Chantaje por fotos o videos suyos en redes sociales'])
Datos['Cantidad de estudiantes'] = 1
Datos['Victimas de abuso físico'] = Datos['Reportes de abuso físico'].apply(lambda x: 1 if x > 0 else 0)
Datos['Victimas de abuso psicológico'] = Datos['Reportes de abuso psicológico'].apply(lambda x: 1 if x > 0 else 0)
Datos['Victimas de ciberbullying'] = Datos['Reportes de ciberbullying'].apply(lambda x: 1 if x > 0 else 0)

# Clasificamos de forma mas especifica el grupo familiar

In [11]:
def clasificar_grupo_familiar(valor):
    if 'Madre' in valor and 'Padre' in valor:
        return 'Ambos'
    elif 'Madre' not in valor and 'Padre' not in valor:
        return 'Ninguno'
    elif 'Madre' in valor and 'Padre' not in valor:
        return 'Solo madre'
    elif 'Padre' in valor and 'Madre' not in valor:
        return 'Solo padre'

Datos['Padres'] = Datos.copy()['Grupo familiar'].apply(clasificar_grupo_familiar)
Datos = Datos.drop(columns=['Grupo familiar'])

# Creamos un documento con la información que se pueda leer en otros notebooks.

In [12]:
Datos.to_pickle('Datos.pkl')